In [8]:
# import all 
def import_all(): 
    import os
    import requests
    import json
    from dotenv import load_dotenv
    import pandas as pd
    import geopandas as gpd
    from cartoframes.viz import Map, Layer, popup_element


In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [2]:
# connect w API 
where1= 'Singapore, Singapore'
where2= 'Zurich, Switzerland '
where3= 'Atlanta, United States'

# connect w API 
url_geocode1 = f"https://geocode.xyz/{where1}?json=1"
res = requests.get(url_geocode1).json()
res

{'success': False,
 'error': {'code': '006',
  'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing',
  'requests': '6'}}

In [3]:
# Get coordinates for 3 wheres  
def get_coordinates (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"

    try:
        res = requests.get(url_geocode).json()
        return res["latt"], res["longt"]
    except:
        print(f"Sorry, no matches found for {where}")
    get_coordinates (where)

In [10]:
# get Coordinates for each city 
Singapore= get_coordinates ('Singapore, Singapore')
Singapore

('1.33594', '103.83717')

In [9]:
Zurich= get_coordinates ('Zurich, Switzerland')
Zurich

('47.36670', '8.55000')

In [8]:
Atlanta = get_coordinates ('Atlanta, United States of America')
Atlanta

('33.77729', '-84.39015')

In [11]:
#put coordinates into Type Point
def type_point(c_tuple):
    TP_coords= {'type': 'Point', 'coordinates': c_tuple}
    return TP_coords

In [13]:
Sing= type_point(Singapore)
Zur= type_point(Zurich)
At= type_point(Atlanta)

{'type': 'Point', 'coordinates': ('33.77729', '-84.39015')}

In [17]:
# Connect w/ Foursquare API 
load_dotenv()
token_fsq = os.getenv("token3")

In [163]:

def get_query(query, coords, limit= 1,radius= 1000):
    """
    Query foursquare around coordinates using outputs from get_coordinates
    """
    lat= coords[0]
    lon= coords[1]
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={str(radius)}&limit={str(limit)}"
    headers = {
        "accept": "application/json",
        "Authorization": token_fsq}
    response = requests.get(url, headers=headers).json()
    return response


In [287]:
def get_all(output_get_query): 
    """
    Get name, address, and geopoints from each city
    """
    new_list= []
    for i in output_get_query["results"]:
        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        category= i['categories'][0]['name']

        type_ = {"typepoint": 
                                {"type": "Point", 
                                "coordinates": [lat, lon]}}
        new_list.append({"name":name, "lat":lat, "lon":lon, "category": category, "type":type_})
    df = pd.DataFrame(new_list)
    return df

In [219]:
# Want df of tech companies, design companies, starbucks, etc for each city

In [376]:
#query API for tech in Singapore
outputsing=get_query('tech',Singapore, limit= 50, radius= 7000)

In [377]:
# Get tech businesses Singapore 
techs= get_all(outputsing)
tech_singapore= techs[techs['category']== 'Technology Business']


In [378]:
#query API for tech in Atlanta 
outputAt=get_query('tech',Atlanta, limit= 50, radius= 7000)

In [379]:
#Get tech businesses Atlanta 
techat= get_all(outputAt)
tech_atlanta= techat[techat['category']== 'Technology Business']


In [380]:
# Query API for tech in Zurich
outputZur=get_query('tech',Zurich, limit= 50, radius= 7000)

In [381]:
# Get tech businesses Zurich
techzur= get_all(outputZur)
tech_Zurich= techzur[techzur['category']== 'Technology Business']


 ### Make Map of Qualities in Singapore 


In [383]:
# Want to add layer to map of starbucks 
desing= get_query('design', Singapore, limit= 50, radius= 7000)

In [444]:
# Want to add layer to map of design companies  
designsing= get_all(desing)

In [385]:
starsing= get_query('Starbucks', Singapore, limit= 50, radius= 7000)

In [390]:
starbucksing= get_all(starsing)

In [393]:
#Query airports Singapore
airsing= get_query('Airport', Singapore, limit= 50, radius= 7000)

In [394]:
airsing= get_all(airsing)

In [407]:
# Get all category airports 
AirSingport= airsing[airsing['category']== 'Airport']
AirSingAll= airsing[airsing['category']== 'Airport Service']
AirSing= pd.concat([AirSingport, AirSingAll], ignore_index=True)

In [411]:
basketsing= get_query('Basketball', Singapore, limit= 50, radius= 7000)

In [437]:
# Drop unwanted Basketball categories 
Basketsing= get_all(basketsing)
BasketSingC= Basketsing[Basketsing['category']!= 'Organization']
BasketSingC.drop(index= 42, axis=1, inplace= True)
BasketSingC.drop(index= 49, axis=1, inplace= True )

In [455]:
# get Daycares for Singapore 
caresing= get_query('daycare', Singapore, limit= 50, radius= 7000)

In [465]:
# Drop non-related queries 
caresingg= get_all(caresing)
caresingg.drop(index= 21, axis= 1, inplace= True)
caresingg.drop(index= 15, axis=1, inplace= True)
caresingg.drop(index= 18, axis=1, inplace= True)
caresingg.drop(index= 19, axis=1, inplace= True)
caresingg.drop(index= 22, axis=1, inplace= True)

In [ ]:
# Get bars from Singapore 
barsing= get_query('bars', Singapore, limit= 50, radius= 7000)
BarSing= get_all(barsing)

In [497]:
#Get dog groomers from Singapore 
groom= get_query('Pet Service', Singapore, limit= 50, radius= 7000)
groomsing= get_all(groom)

In [510]:
GroomSing= groomsing[groomsing['category']== 'Pet Grooming Service']
GroomSing.reset_index(drop=True, inplace=True)

In [512]:
# Make singapore map with layers 
Sing_geodf = gpd.GeoDataFrame(tech_singapore, geometry=gpd.points_from_xy(tech_singapore["lon"], tech_singapore["lat"]))
Sing_geoStar= gpd.GeoDataFrame(starbucksing, geometry=gpd.points_from_xy(starbucksing["lon"], starbucksing["lat"]))
Sing_geoDes= gpd.GeoDataFrame(designsing, geometry=gpd.points_from_xy(designsing["lon"], designsing["lat"]))
Sing_geoAir= gpd.GeoDataFrame(AirSing, geometry=gpd.points_from_xy(AirSing["lon"], AirSing["lat"]))
Sing_geoBas= gpd.GeoDataFrame(BasketSingC, geometry=gpd.points_from_xy(BasketSingC["lon"], BasketSingC["lat"]))
Sing_geoCare= gpd.GeoDataFrame(caresingg, geometry=gpd.points_from_xy(caresingg["lon"], caresingg["lat"]))
Sing_geoBar= gpd.GeoDataFrame(BarSing, geometry=gpd.points_from_xy(BarSing["lon"], BarSing["lat"]))
Sing_geoGroom= gpd.GeoDataFrame(GroomSing, geometry=gpd.points_from_xy(GroomSing["lon"], GroomSing["lat"]))




In [513]:

Map([
    Layer(Sing_geodf, "color:blue", popup_hover=[popup_element("name")]),
    Layer(Sing_geoStar, 'color:green', popup_hover=[popup_element("name")]),
    Layer(Sing_geoDes, 'color:purple', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoAir, 'color:white', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoBas, 'color:orange', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoCare, 'color:pink', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoBar, 'color:black', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoGroom, 'color:brown', popup_hover=[popup_element("name")])
])


,name,lat,lon,category,type
0,Starbucks,1.322263,103.844340,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Starbucks,1.349750,103.850687,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Starbucks,1.332478,103.848648,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Starbucks,1.319955,103.844378,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Starbucks,1.304850,103.831219,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Starbucks,1.307056,103.828497,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Starbucks Reserve,1.317673,103.844021,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Starbucks,1.301071,103.862711,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Starbucks,1.304462,103.831595,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,Starbucks,1.300767,103.839336,Coffee Shop,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [305]:
import pymongo
client = pymongo.MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
# connect w MongoDB 
db = client.get_database("Ironhack")
nyrest = db.get_collection("nyrest")
nyneigh = db.get_collection("nyneigh")

In [ ]:
# poor forgotten function 
def format_coors(og_tuple):
    """
    Format outputs from get_coordinates to be used in get_query
    """
    listt= list(og_tuple)
    listfloats= [float(x) for x in listt]
    roundedlist= [round(x, 2) for x in listfloats]
    roundedstring= str(roundedlist)
    cleaned= roundedstring[1:-1]
    return cleaned
